# pystac: Working with catalogs

##### Example catalog structrure:
```
example-catalog/
├── catalog.json
└── eo
    ├── catalog.json
    ├── landsat-8-l1
    │   ├── catalog.json
    │   └── item.json
    └── sentinel-2-l1c
        ├── catalog.json
        └── sentinel-2a
            ├── catalog.json
            └── item.json
```

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
from os.path import dirname
import pprint

from pystac import Catalog, Collection, Item

In [4]:
pp = pprint.PrettyPrinter(indent=4)

### Working with existing catalogs

In [ ]:
# open a root catalog
cat = Catalog.from_file('example-catalog/catalog.json')

In [ ]:
# get first (and only in this case) sub-catalog
subcat = cat.get_children()[0]

In [ ]:
# print some IDs
print("Root Catalog: ", cat.id)
print("Sub Catalog: ", subcat.id)
print("Sub Catalog parent: ", subcat.get_parent().id)

# iterate through child catalogs of the sub-catalog
print("Sub Catalog children:")
for child in subcat.get_children():
    print('    ', child.id)

In [ ]:
all_catalogs = cat.catalogs()
all_collections = cat.collections()
all_items = cat.items()

print(all_catalogs)
print(all_collections)
print(all_items)

In [ ]:
print('**Catalogs**')
for c in cat.catalogs():
    print(c.id)

print('\n**Collections**')
for c in cat.collections():
    print(c.id)
    
print('\n**Items**')
for i in cat.items():
    print(i.id)

### Creating new catalogs

In [5]:
# create a Catalog object with JSON
cat_json = {
    "id": "mycat",
    "description": "My shiny new STAC catalog",
    "href": "my_cat"
}
mycat = Catalog(**cat_json)

In [6]:
mycat.set_uris_from_root('my_cat')
mycat.save()

In [7]:
print(mycat.id)
# the filename is then stored with the object
filename = mycat.get_links('self')[0].target
print(filename)
path = dirname(filename)
print(path)

mycat
my_cat/catalog.json
my_cat


### Adding catalogs to catalogs

In [8]:
# add a new catalog to a root catalog
kitten_json = {
    "id": "mykitten",
    "description": "A child catalog of my shiny new STAC catalog"
}

kitten = Catalog(**kitten_json)
mycat.add_child(kitten)

kitten.set_relative_to_parent()
print('Child catalog filename after adding: ', kitten.get_single_link('self').target)

Child catalog filename after adding:  my_cat/mykitten/catalog.json


In [9]:
mycat.save()

### Adding collections to catalogs

In [11]:
# open the Landsat collection
collection = Collection.from_file('example-catalog/eo/landsat-8-l1/catalog.json')
print('Collection name: ', collection.id)

Collection name:  landsat-8-l1


In [13]:
collection.set_root(mycat)

In [14]:
collection.links

[<Link rel=parent target=../catalog.json>,
 <Link rel=item target=item.json>,
 <Link rel=self target=example-catalog/eo/landsat-8-l1/catalog.json>,
 <Link rel=root target=<Catalog id=mycat>>]

In [15]:
collection.set_parent(kitten)

In [17]:
collection.links

[<Link rel=item target=item.json>,
 <Link rel=self target=example-catalog/eo/landsat-8-l1/catalog.json>,
 <Link rel=root target=<Catalog id=mycat>>,
 <Link rel=parent target=<Catalog id=mykitten>>]

In [18]:
collection.clear_links('item')

In [19]:
collection.links

[<Link rel=self target=example-catalog/eo/landsat-8-l1/catalog.json>,
 <Link rel=root target=<Catalog id=mycat>>,
 <Link rel=parent target=<Catalog id=mykitten>>]

In [20]:
collection.set_relative_to_parent()

In [21]:
collection.links

[<Link rel=root target=<Catalog id=mycat>>,
 <Link rel=parent target=<Catalog id=mykitten>>,
 <Link rel=self target=my_cat/mykitten/landsat-8-l1/collection.json>]

In [22]:
# add it to the child catalog created above
kitten.add_child(collection)
print('Collection filename: ', collection.filename)

Collection filename:  my_cat/mykitten/landsat-8-l1/collection.json


In [23]:
mycat.save()

> /home/simon/files/rv/stac/pystac/pystac/io.py(56)save_json()
-> os.makedirs(dirname)
(Pdb) c


In [ ]:
print('\n**Collection links**')
pp.pprint(collection.get_links())

In [ ]:
collection.set_relative_paths()

In [ ]:
collection.full_copy(mycat, kitten)

In [ ]:
item.set_parent(kitten)

In [ ]:
item.links

### Adding items to collection

In [ ]:
# open a Landsat item
item = Item.from_file('example-catalog/eo/landsat-8-l1/item.json')
print('Item name: ', collection)

# add it to the collection created above
collection.add_item(item)
# print('Item filename: ', item.filename)

# print('\n**Item links**')
# pp.pprint(item._data['links'])

In [ ]:
item.get_self_href()

In [ ]:
collection.get_self_href()

In [ ]:
item.set_uris_

In [ ]:
item.get_self_href()

In [ ]:
collection.set_relative_to_parent()

In [ ]:
item.get_self_href()